In [4]:
import pandas as pd
import json
import numpy as np
import requests
from config import (client_id, api_key)
from pprint import pprint
import pydbgen
from pydbgen import pydbgen
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
business_data={"Business ID":[],
               "Business Name":[],
               "Business Alias":[],
               "Category":[],
               "City":[],
               "State":[],
               "ZipCode":[],
               "Lat":[],
               "Long":[],
               "Review Count":[],
               "Rating":[],
              }
review_data={"Business ID":[],
             "Review":[],
            }

In [6]:
myDB=pydbgen.pydb()
city=[]
for cities in range(500):
    city.append(myDB.city_real())
city_yelp=pd.DataFrame(city)
city_yelp=city_yelp.rename(columns={0:"US Cities"})
city_yelp.head()

,US Cities
0,Dunes City
1,Minam
2,Slaughterville
3,Breidablick
4,Pellettown


In [7]:
api_key=api_key
headers = {"Authorization": (f"Bearer {api_key}")}
url="https://api.yelp.com/v3/businesses/search?"
review_url="https://api.yelp.com/v3/businesses/"
categories="lawyers"
radius = 40000
counter = 1

In [8]:
for index, row in city_yelp.iterrows():
    try:
        random_city = row['US Cities']
        print(f"{counter}: {random_city}")
        yelp_response2 = requests.get((f"{url}categories={categories}&location={random_city}&radius={radius}"), headers=headers).json()
        #pprint(yelp_response2)
        counter+=1
    except:
        print(f"Could not locate any lawyers in {random_city}.")
    #print(len(yelp_response2["businesses"]))
    try:    
        for x in range(len(yelp_response2["businesses"])):
            try:
                business_data["Business ID"].append(yelp_response2["businesses"][x]["id"])
                business_data["Business Name"].append(yelp_response2["businesses"][x]["name"])
                business_data["Business Alias"].append(yelp_response2["businesses"][x]["alias"])
                business_data["Category"].append(yelp_response2["businesses"][x]["categories"][0]["alias"])
                business_data["City"].append(yelp_response2["businesses"][x]["location"]["city"])
                business_data["State"].append(yelp_response2["businesses"][x]["location"]["state"])
                business_data["ZipCode"].append(yelp_response2["businesses"][x]["location"]["zip_code"])
                business_data["Lat"].append(yelp_response2["businesses"][x]["coordinates"]["latitude"])
                business_data["Long"].append(yelp_response2["businesses"][x]["coordinates"]["longitude"])
                business_data["Review Count"].append(yelp_response2["businesses"][x]["review_count"])
                business_data["Rating"].append(yelp_response2["businesses"][x]["rating"])
            except: 
                print(f"     No law offices found in {random_city} using Yelp.com.")
    except:
        print(f"     No law offices found in {random_city} using Yelp.com.")

1: Dunes City
2: Minam
3: Slaughterville
4: Breidablick
5: Pellettown
6: Tamarac
7: Susanville
8: Gulf Crest
9: Combes
10: Jolly
11: Du Quoin
12: Taylorsport
13: Pahrump
14: Crow Agency
15: Timonium
16: Childersburg
17: Zona
18: Munday
19: Rickardsville
20: Jenifer
21: South Lineville
22: Lawton
23: Kahua
24: Fireco
25: Powell
26: Joyce
27: Wray
28: Waimanalo Beach
29: Shady Grove
30: Soledad
31: Timberlane
32: Montgomery Creek
33: Loranger
34: Bascom
35: Plata
36: Young America
37: Spaulding
38: Wisacky
39: Madeline
40: Hicksville
41: East Quogue
42: Crown Point
43: Taylors
44: Idleyld Park
45: McMinnville
46: Bragg City
47: Honda
48: Jarbidge
49: Lawtell
50: Stimson Crossing
51: Mousie
52: Canova
53: Sabinoso
54: Faith
55: Lille
56: Bushkill
57: Glass
58: Glen Allen
59: Elk Mills
60: Rural Retreat
61: Lazy Lake
62: Sunderland
63: Oak Ridge
64: Hill City
65: Green Hill
66: Aliceville
67: Monona
68: Elk Plain
69: Kline
70: Devereux
71: Mocanaqua
72: Carnesville
73: Wallkill
74: Armbrus

In [ ]:
#pprint(yelp_response2)

In [ ]:
#business_data

In [ ]:
graph_data=pd.DataFrame.from_dict(business_data, orient='columns', dtype=None, columns=None)

In [ ]:
graph_data["Review Count"].sum()

In [ ]:
for id in business_data["Business ID"]:
    try:
        yelp_response3 = requests.get((f"{review_url}{id}/reviews"), headers=headers).json()
        #pprint(yelp_response3)
        print(f"Fetching Reviews for {id}.")
    except:
        print(f"No Reviews for {id}.")
    try:
        for x in range(len(yelp_response3["reviews"])):
            try:
                review_data["Business ID"].append(id)
                review_data["Review"].append(yelp_response3["reviews"][x]["text"])
            except:
                print("skipping")
    except:
        print("skipping")

In [ ]:
graph_review=pd.DataFrame.from_dict(review_data, orient='columns', dtype=None, columns=None)

In [ ]:
graph_review["Compound"]=""
graph_review.head()

for index, row in graph_review.iterrows():
    results = analyzer.polarity_scores(row["Review"])
    graph_review.loc[index, "Compound"] = results["compound"]
graph_review.head()

In [ ]:
final=graph_review.merge(graph_data, on="Business ID", how="left")
final.head(100000)

In [ ]:
final.to_csv(path_or_buf="Merged3.csv", sep=',')